In [95]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import time, json, requests

from server import *

In [ ]:
nickname_list = []

# 웹드라이버 설정
driver = webdriver.Chrome()  # 크롬 드라이버 경로 설정
driver.get('https://fconline.nexon.com/datacenter/rank')

time.sleep(3)  # 페이지 로딩 대기

current_page = 1

while True:
    try:
        # 현재 페이지 처리
        print(f"Processing page {current_page}")

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        nickname_list.append(soup.find_all('span', class_ = 'name profile_pointer'))
        
        # 맨 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 페이지 안정화 대기
        
        # 다음 페이지 버튼 찾기 (동적 로딩 대기)

        if current_page % 10 == 0:
            
            
            next_page = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@class="btn_next_list ajaxNav"]'))
            )
            print('다음 페이지로 넘어갑니다.')

        else:

            next_page = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, f'//a[@class="btn_num ajaxNav" and text()="{current_page + 1}"]'))
            )

        # 다음 페이지 클릭
        next_page.click()
        time.sleep(3)  # 페이지 로딩 대기

        # 페이지 번호 업데이트
        current_page += 1

    except Exception as e:
        print("No more pages or an Error occurred:", e)
        break

driver.quit()

Processing page 1
Processing page 2
Processing page 3
Processing page 4
Processing page 5
Processing page 6
Processing page 7
Processing page 8
Processing page 9
Processing page 10
다음 페이지로 넘어갑니다.
Processing page 11
Processing page 12
Processing page 13
Processing page 14
Processing page 15
Processing page 16
Processing page 17
Processing page 18
Processing page 19
Processing page 20
다음 페이지로 넘어갑니다.
Processing page 21
Processing page 22
Processing page 23
Processing page 24
Processing page 25
Processing page 26
Processing page 27
Processing page 28
Processing page 29
Processing page 30
다음 페이지로 넘어갑니다.
Processing page 31
Processing page 32
Processing page 33
Processing page 34
Processing page 35
Processing page 36
Processing page 37
Processing page 38
Processing page 39
Processing page 40
다음 페이지로 넘어갑니다.
Processing page 41
Processing page 42
Processing page 43
Processing page 44
Processing page 45
Processing page 46
Processing page 47
Processing page 48
Processing page 49
Processing page 50

In [74]:
nickname_list

[[],
 [<span class="name profile_pointer" data-sn="1139420599">Prime제휘담</span>,
  <span class="name profile_pointer" data-sn="200197430">SainT성지원</span>,
  <span class="name profile_pointer" data-sn="1223257130">카메라</span>,
  <span class="name profile_pointer" data-sn="585491786">Innate김승환</span>,
  <span class="name profile_pointer" data-sn="703232217">Climax빌런</span>,
  <span class="name profile_pointer" data-sn="1676363999">노영진짜못해</span>,
  <span class="name profile_pointer" data-sn="1290366125">UnLMTD원재</span>,
  <span class="name profile_pointer" data-sn="1626005060">Prime정민</span>,
  <span class="name profile_pointer" data-sn="703152739">리바이브인호</span>,
  <span class="name profile_pointer" data-sn="1793528007">SainT지원</span>,
  <span class="name profile_pointer" data-sn="1910598220">DHFC김병권</span>,
  <span class="name profile_pointer" data-sn="870562747">Prime림광철</span>,
  <span class="name profile_pointer" data-sn="1072028533">내가왜져야하는데</span>,
  <span class="name profile_pointer"

In [ ]:
li = []

for page in nickname_list:
    
    for span in page:

        nickname = span.get_text() # 닉네임만 추출

        li.append(nickname)

In [ ]:
ouid_dic = {}
func = USER_FUNCTION['id'] # ouid 조회

for nn in li:

    urlString = basic_link + func + '?nickname=' + nn

    response = requests.get(urlString, headers = headers)
    result = response.json()['ouid']

    ouid_dic[nn] = result

In [ ]:
# JSON 파일로 저장
with open("data/user_ouid.json", "w", encoding="utf-8") as file:
    
    json.dump(ouid_dic, file, ensure_ascii=False, indent=4)  # ensure_ascii=False로 한글 깨짐 방지